In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime
import pandas as pd
plt.style.use('fivethirtyeight')
pd.options.mode.chained_assignment = None  # default='warn'
import collections
import multiprocessing

In [ ]:
def SMA(data, period = 30, column='<CLOSE>'):
  return data[column].rolling(window=period).mean()

In [ ]:
def SMA_fun(data, Q, S):
  Profit = 0
  Loss = 0
  data['SMA_Q'] = SMA(data, Q)
  data['SMA_S'] = SMA(data, S)
  data['Signal'] = np.where(data['SMA_Q'] > data['SMA_S'], 1, 0)
  data['Position'] = data['Signal'].diff()
  data['Buy'] = np.where(data['Position'] == 1, data['<CLOSE>'], -1)
  df_new = data[['<LOW>', '<HIGH>', 'Buy']]
  BuyList = df_new[data['Buy'] != -1]
  LIST = list(BuyList.index)
  for i in LIST:
    a = BuyList.loc[i]['Buy']
    TakeProfit = df_new['<HIGH>'] / a >= 1.01
    StopLoss = df_new['<LOW>'] / a <= 0.99
    df_new['Result'] = 2 * TakeProfit - 1 * StopLoss
    B1 = np.trim_zeros(df_new.loc[i:]['Result'])
    if len(B1) != 0:
      if B1[0] > 0:
        Profit += 1
      else:
        Loss += 1
  print(Q, S, Profit-Loss)
  return (Profit, Loss)

In [ ]:
def process_item(item):
     return {'Q': item.Q, 'S': item.S, 'score' : SMA_fun(df, item.Q, item.S) }

In [ ]:
for name in ['YNDX_200101_200331', 'YNDX_200401_200630', 'YNDX_200701_200930', 'YNDX_201001_201231']:
 start_time = datetime.now()
 FILE_NAME = name + '.csv'
 df = pd.read_csv(FILE_NAME)
 df = df.set_index(pd.DatetimeIndex(df['<DATE>'].values + ' ' + df['<TIME>'].values))
 df_result = pd.DataFrame({'Quik' : [], 'Slow' : [], 'Result' : []})
 res = collections.namedtuple('res', ['Q', 'S', 'score'])
 variants = []
 for i in range(1, 151):
   for j in range(i+1, 151):
     variants.append(res(Q=i, S = j, score = 0))
 pool = multiprocessing.Pool()
 result = pool.map(process_item, variants)
 df_result = pd.DataFrame({'Quik' : [], 'Slow' : [], 'Result' : []})
 for i in tuple(result):
   df_result = df_result.append({'Quik' : i['Q'], 'Slow' : i['S'], 'Result' :  i['score']}, ignore_index = True)
 df_result.to_csv(name + '_result' + '.csv')
 df_result.to_excel(name + '_result' + '.xlsx')
 print(datetime.now() - start_time)

Выходные данные были обрезаны до нескольких последних строк (5000).
34 121 39
34 122 37
10 43 110
34 123 34
34 124 30
34 125 28
10 44 116
34 126 23
34 127 29
34 128 26
10 45 116
34 129 29
34 130 33
34 131 32
10 46 104
34 132 29
34 133 32
34 134 29
34 135 26
10 47 117
34 136 21
34 137 21
34 138 28
10 48 112
34 139 33
34 140 30
34 141 21
10 49 115
34 142 22
34 143 24
34 144 20
34 145 19
10 50 113
34 146 16
34 147 14
34 148 12
10 51 102
34 149 13
34 150 15
10 52 99
10 53 90
10 54 86
10 55 84
35 36 414
10 56 79
10 57 78
35 37 269
10 58 71
10 59 78
10 60 74
35 38 215
10 61 78
10 62 80
35 39 183
10 63 74
10 64 77
35 40 170
10 65 79
35 41 140
10 66 76
10 67 74
35 42 134
10 68 72
35 43 137
10 69 72
10 70 75
35 44 131
10 71 69
35 45 125
10 72 63
35 46 115
10 73 70
10 74 67
35 47 103
10 75 71
35 48 95
10 76 73
35 49 75
10 77 68
35 50 57
10 78 60
35 51 59
10 79 53
10 80 41
35 52 54
10 81 47
35 53 56
10 82 49
35 54 46
10 83 53
35 55 60
10 84 44
35 56 67
10 85 45
35 57 64
10 86 42
35 58 68
10 87 39